In [1]:
import sys
from vispy import app, scene, visuals,use
from vispy.util.filter import gaussian_filter
import numpy as np
import vispy.io as io
from vispy.scene.visuals import Ellipse
# if offscreen 
# use egl or osmesa
import filepath
import os
from util.repo import res_root
from util.texture import np2texture, rgba_mpl2pd3d, texture_load_np
from panda3d.core import PNMImage, Texture
from panda3d_game.app import *
from demos.ball_room import MassedBall,tmoon
from panda3d.core import (
    Geom,
    GeomNode,
    GeomTriangles,
    GeomVertexData,
    GeomVertexFormat,
    GeomVertexWriter,
    GeomEnums
)
from panda3d.core import (
    NodePath,
    CardMaker,
    Texture,
    CardMaker,
    Point2,
    Vec3
)
from vispy.visuals.transforms.linear import MatrixTransform
from art.basic import create_cylinder_node, create_sphere_node
# import direct.directbase.DirectStart
from util.physics import autocomplete_units, G_val, getG
from panda3d.core import AmbientLight
from sympy.physics.units import (
    kilometer, meter,centimeter,
    gram, kilogram, tonne,
    newton, second
)
from util.rendering import *
import traceback
try:
    use( 'glfw')
except Exception as e:
    print(e)
    print(traceback.format_exc())
    pass
class StarCanvas(scene.SceneCanvas):
    def __init__(self, n_stars, rho=10):
        scene.SceneCanvas.__init__(
            self,keys='interactive', title='Isocurve(s) overlayed '
                           'over Random Image Example',show=False,
            # size=(1000,500)
        )
        # self.size =  800, 1600
        self.unfreeze()
        # Set up a viewbox to display the image with interactive pan/zoom
        self.view = self.central_widget.add_view()
        self.view.camera = scene.TurntableCamera() # so it is 2d
        self.view.camera.set_range()
        self.n_stars = n_stars 
        # self.view = self.central_widget.add_view()
        # self.view.camera = scene.PanZoomCamera(aspect=1) # so it is 2d
        np.random.seed(0)  # todo: use torch random state
        cos_pos_theta = np.random.uniform(-1,1, n_stars)
        pos_theta = np.arccos(cos_pos_theta)
        pos_phi = np.random.uniform(0,2*np.pi, n_stars)
        sizes = np.random.uniform(0, 5, n_stars) 
        # h = sizes
        # w = sizes / np.sin(pos_theta)
        # centers = np.vstack([pos_phi,pos_theta]).T
        r = rho * np.sin(pos_theta)
        x = r * np.cos(pos_phi)
        y = r * np.sin(pos_phi)
        z = rho * np.cos(pos_theta)
        positions = np.vstack([x,y,z]).T
        scatter = scene.visuals.Markers()
        scatter.set_data(positions, edge_color=None, face_color='white', size=sizes, symbol='o')
        # Add scatter plot to the view
        self.view.add(scatter)


    def update_camera(self, mat: np.ndarray, fov=None, aspect=None):
        self.view.scene.transform = MatrixTransform(
            mat
        )
        # self.view.camera.orbit(1,0)
        if fov is not None:
            self.view.camera.fov=fov
        if aspect is not None:
            self.view.camera.aspect = aspect
        self.update()

    def update_camera_hpr(self, h=0,p=0,r=0,fov=None, aspect=None):
        self.view.camera.azimuth = h
        self.view.camera.elevation = p
        self.view.camera.roll = r
        if fov is not None:
            self.view.camera.fov=fov
        if aspect is not None:
            self.view.camera.aspect = aspect
        self.update()



class StarSphere(ControlShowBase):
    def __init__(self):
        ContextShowBase.__init__(self)
        ControlShowBase.__init__(self)
        self.stars_canvas = StarCanvas(60000)
        self.stars_canvas.app.run()
        self.bg_texture = Texture()
        self.bg_texture.set_wrap_u(Texture.WM_clamp)
        self.bg_texture.set_wrap_v(Texture.WM_clamp)
        self.bg_cm = CardMaker('bg')
        self.bg_cm.setFrame(-1,1,-1,1)
        self.bg_initial_width = 2
        self.bg_initial_height = 2
        self.bg_cm.setUvRange(Point2(0, 1),  # ll
                          Point2(1, 1),  # lr
                          Point2(1, 0),  # ur
                          Point2(0, 0))
        self.bg_plane_pth = self.display_camera.attach_new_node(self.bg_cm.generate())
        # self.bg_plane_pth = self.rdr_scene.attach_new_node(self.bg_cm.generate())
        # self.bg_plane_pth.reparentTo(self.display_camera)
        self.bg_near = 10
        top, right = self.bg_fit_lens()
        
        self.bg_plane_pth.setPos(0,self.bg_near,0)
        # self.display_camera.reparentTo(self.rdr_scene)
        # self.bg_plane.reparentTo(self.render)
        # self.bg_plane_pth.setColor((1,1,1,1))
        self.bg_plane_pth.setTexture(self.bg_texture)
        self.bg_plane_pth.setBin("background", SORT_BG)
        self.bg_plane_pth.setDepthTest(False)
        self.bg_plane_pth.setDepthWrite(False)
        
        self.taskMgr.add(self.updateStarsTask)

    def updateStarsTask(self, task):
        top, right = self.bg_fit_lens()
        # cam_mat = np.array(self.cam.get_mat())
        # self.stars_canvas.update_camera(cam_mat)
        self.stars_canvas.update_camera_hpr(
            self.display_camera.getH(),
            -self.display_camera.getP(),
            self.display_camera.getR(),
        )
        stars = self.stars_canvas.render()
        texture_load_np(self.bg_texture, stars, format_=Texture.F_rgba)
        # TODO: let card to follow camera
        
        return task.cont

    def bg_fit_lens(self):
        lens = self.camLens
        fov_rad = lens.getFov() * (np.pi/180)
        # print(lens.getFov)
        aspect_ratio = lens.getAspectRatio()
        
        top = self.bg_near * np.tan(fov_rad[1]/2)*2
        right = self.bg_near * np.tan(fov_rad[0]/2)*2
        scale = max(top/self.bg_initial_height, right/self.bg_initial_width)
        # print(top,right)
        self.bg_plane_pth.setScale(scale,1,scale)
        return top, right
    

class PlanetSphere(StarSphere,PhysicsShowBase):
    def __init__(self):
        
        StarSphere.__init__(self)
        PhysicsShowBase.__init__(self)
        self.bullet_world.setGravity((0,0,0))
        self.unit = {
            "mass" : tonne,
            "length" : 100*meter,
            "time": 1 * second,
            # "force" : sp.Number(1e3) * newton
        }
        autocomplete_units(self.unit)
        self.planet1 = MassedBall(
            name="planet1",
            radius=100*meter,
            mass=1e6*tonne,
            units=self.unit
        )
        self.planet1.reparentTo(self.render)
        self.planet1.set_texture(tmoon)
        self.planet1.setScale(10)
        self.planet1.setPos(0,10,0)

if __name__ == "__main__":
    from ui.abstract_ui import InterfacePlaceHolder
    import builtins
    import traceback
    # interface = InterfacePlaceHolder()
    try:
        with PlanetSphere() as app_:
            # console = PhyscRoomConsole(showbase=app)
            # interface = CMDInterface(console=console)
            # .start()
            # start a thread of app
            app_.run()
    except Exception as e:
        print(e)
        print(traceback.format_exc())
    finally:
        if hasattr(builtins, 'base'):
            builtins.base.destroy()
        # interface.join()


Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


init ContextShowBase
---set ref---:render,<class 'panda3d.core.NodePath'>
---<__main__.PlanetSphere object at 0x7f30b8944650> run(), at 2024-10-06 03:52:12.477518---


/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1711322746054/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


TypeError: float() argument must be a string or a real number, not 'panda3d.core.LVecBase2f'

---<__main__.PlanetSphere object at 0x7f30b8944650> destroy at 2024-10-06 03:52:12.584118---
---<__main__.PlanetSphere object at 0x7f30b8944650> destroyed at 2024-10-06 03:52:12.586300, exit---
float() argument must be a string or a real number, not 'panda3d.core.LVecBase2f'
Traceback (most recent call last):
  File "/tmp/ipykernel_141823/2359208593.py", line 208, in <module>
    app_.run()
  File "/media/ywatcher/ExtDisk1/Files/Games/learn_panda3d/p1/py_src/panda3d_game/app/app_.py", line 76, in run
    super().run()
  File "/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/direct/showbase/ShowBase.py", line 3333, in run
    self.taskMgr.run()
  File "/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/direct/task/Task.py", line 553, in run
    self.step()
  File "/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/direct/task/Task.py", line 504, in step
    self.mgr.poll()
  File "/tmp/ipykern

:task(error): Exception occurred in PythonTask StarSphere.updateStarsTask


In [22]:
type(canvas.view.camera.transformii)

vispy.visuals.transforms.linear.MatrixTransform

In [25]:
vispy.visuals.transforms.linear.MatrixTransform(np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,0],
    [0,0,0,1]
]))

MatrixTransform(matrix=[[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 1, 0],
                        [0, 0, 0, 1]] at 0x7f3442ce7850)

In [5]:
import vispy
import panda3d